## Vector Database setup

Remove old Weaviate DB files

In [1]:
!rm -rf ~/.local/share/weaviate


### Step 1 - Download sample data

In [6]:
import os
import requests
import json

from dotenv import load_dotenv, find_dotenv
import openai
import weaviate
from weaviate import EmbeddedOptions

In [3]:
# Download the data
resp = requests.get(
    'https://raw.githubusercontent.com/weaviate-tutorials/'
    'quickstart/main/data/jeopardy_tiny.json')
data = json.loads(resp.text)  # Load data
# Parse the JSON and preview it
print(type(data), len(data))

<class 'list'> 10


In [4]:
def json_print(data):
    print(json.dumps(data, indent=2))

In [5]:
json_print(data[0])

{
  "Category": "SCIENCE",
  "Question": "This organ removes excess glucose from the blood & stores it as glycogen",
  "Answer": "Liver"
}


### Step 2 - Create an embedded instance of Weaviate vector database

In [7]:
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
client = weaviate.Client(
    embedded_options=EmbeddedOptions(),
    additional_headers={
        "X-OpenAI-BaseURL": os.environ['OPENAI_API_BASE'],
        # Replace this with your actual key
        "X-OpenAI-Api-Key": openai.api_key  })
print(f"Client created? {client.is_ready()}")

Binary /home/jovyan/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.22.3/weaviate-v1.22.3-Linux-amd64.tar.gz
Started /home/jovyan/.cache/weaviate-embedded: process ID 128
Client created? True


{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2023-12-02T19:20:21Z"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2023-12-02T19:20:21Z"}
{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2023-12-02T19:20:21Z"}
{"action":"grpc_startup","level":"info","msg":"grpc server listening at [::]:50060","time":"2023-12-02T19:20:21Z"}
{"action":"restapi_management","level":"info","msg":"Serving weaviate at http://127.0.0.1:8079","time":"2023-12-02T19:20:21Z"}


In [8]:
json_print(client.get_meta())

{
  "hostname": "http://127.0.0.1:8079",
  "modules": {
    "generative-openai": {
      "documentationHref": "https://platform.openai.com/docs/api-reference/completions",
      "name": "Generative Search - OpenAI"
    },
    "qna-openai": {
      "documentationHref": "https://platform.openai.com/docs/api-reference/completions",
      "name": "OpenAI Question & Answering Module"
    },
    "ref2vec-centroid": {},
    "reranker-cohere": {
      "documentationHref": "https://txt.cohere.com/rerank/",
      "name": "Reranker - Cohere"
    },
    "text2vec-cohere": {
      "documentationHref": "https://docs.cohere.ai/embedding-wiki/",
      "name": "Cohere Module"
    },
    "text2vec-huggingface": {
      "documentationHref": "https://huggingface.co/docs/api-inference/detailed_parameters#feature-extraction-task",
      "name": "Hugging Face Module"
    },
    "text2vec-openai": {
      "documentationHref": "https://platform.openai.com/docs/guides/embeddings/what-are-embeddings",
      "nam

## Step 3 - Create Question collection

In [9]:
# resetting the schema. CAUTION: This will delete your 
# collection 
if client.schema.exists("Question"):
    client.schema.delete_class("Question")
class_obj = {
    "class": "Question",
    # Use OpenAI as the vectorizer
    "vectorizer": "text2vec-openai",  
    "moduleConfig": {
        "text2vec-openai": {
            "model": "ada",
            "modelVersion": "002",
            "type": "text",
            "baseURL": os.environ["OPENAI_API_BASE"]}}}
client.schema.create_class(class_obj)

{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"question_00PqQmM4id2z","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2023-12-02T19:22:54Z","took":44891}


## Step 4 - Load sample data and generate vector embeddings

In [10]:
# reminder for the data structure
json_print(data[0])

{
  "Category": "SCIENCE",
  "Question": "This organ removes excess glucose from the blood & stores it as glycogen",
  "Answer": "Liver"
}


In [11]:
with client.batch.configure(batch_size=5) as batch:
    for i, d in enumerate(data):  # Batch import data
        print(f"importing question: {i+1}")
        properties = {
            "answer": d["Answer"],
            "question": d["Question"],
            "category": d["Category"]}
        batch.add_data_object(
            data_object=properties,
            class_name="Question")

importing question: 1
importing question: 2
importing question: 3
importing question: 4
importing question: 5
importing question: 6
importing question: 7
importing question: 8
importing question: 9
importing question: 10


In [12]:
count = client.query.aggregate("Question").with_meta_count().do()
json_print(count)

{
  "data": {
    "Aggregate": {
      "Question": [
        {
          "meta": {
            "count": 10
          }
        }
      ]
    }
  }
}


## Let's Extract the vector that represents each question!

In [13]:
# write a query to extract the vector for a question
result = (
    client
    .query
    .get("Question", ["category", "question", "answer"])
    .with_additional("vector")
    .with_limit(1)
    .do())
json_print(result)

{
  "data": {
    "Get": {
      "Question": [
        {
          "_additional": {
            "vector": [
              0.0012505861,
              0.010590901,
              0.028476886,
              -0.022797143,
              -0.025702149,
              0.008474023,
              -0.015554165,
              -0.01608827,
              -0.01100125,
              -0.009972122,
              0.00656232,
              -0.008584753,
              -0.01935803,
              0.007894325,
              -0.004087202,
              -0.016830804,
              0.025128964,
              -0.026835494,
              -0.023370327,
              -0.004445443,
              -0.006425537,
              0.0020826817,
              0.008005054,
              0.0068782233,
              0.024620913,
              -0.0013246768,
              0.01901933,
              -0.03876817,
              -0.0012391875,
              -0.00030084085,
              0.0025369963,
              -0.012818507,
       

## Query time
What is the distance between the `query`: `biology` and the returned objects?

In [14]:
response = (
    client
    .query
    .get("Question",["question","answer","category"])
    .with_near_text({"concepts": "biology"})
    .with_additional('distance')
    .with_limit(2)
    .do())
json_print(response)

{
  "data": {
    "Get": {
      "Question": [
        {
          "_additional": {
            "distance": 0.19695163
          },
          "answer": "DNA",
          "category": "SCIENCE",
          "question": "In 1953 Watson & Crick built a model of the molecular structure of this, the gene-carrying substance"
        },
        {
          "_additional": {
            "distance": 0.20142835
          },
          "answer": "species",
          "category": "SCIENCE",
          "question": "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification"
        }
      ]
    }
  }
}


In [15]:
response = (
    client
    .query
    .get("Question", ["question", "answer"])
    .with_near_text({"concepts": ["animals"]})
    .with_limit(10)
    .with_additional(["distance"])
    .do())
json_print(response)

{
  "data": {
    "Get": {
      "Question": [
        {
          "_additional": {
            "distance": 0.18963969
          },
          "answer": "Elephant",
          "question": "It's the only living mammal in the order Proboseidea"
        },
        {
          "_additional": {
            "distance": 0.19144487
          },
          "answer": "the nose or snout",
          "question": "The gavial looks very much like a crocodile except for this bodily feature"
        },
        {
          "_additional": {
            "distance": 0.20419747
          },
          "answer": "Antelope",
          "question": "Weighing around a ton, the eland is the largest species of this animal in Africa"
        },
        {
          "_additional": {
            "distance": 0.21438634
          },
          "answer": "species",
          "question": "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification"
        },
        {
    

## We can let the vector database know to remove results after a threshold distance!

In [16]:
response = (
    client.query
    .get("Question", ["question", "answer"])
    .with_near_text({"concepts": ["animals"], "distance": 0.24})
    .with_limit(10)
    .with_additional(["distance"])
    .do())
json_print(response)

{
  "data": {
    "Get": {
      "Question": [
        {
          "_additional": {
            "distance": 0.18963969
          },
          "answer": "Elephant",
          "question": "It's the only living mammal in the order Proboseidea"
        },
        {
          "_additional": {
            "distance": 0.19144487
          },
          "answer": "the nose or snout",
          "question": "The gavial looks very much like a crocodile except for this bodily feature"
        },
        {
          "_additional": {
            "distance": 0.20419747
          },
          "answer": "Antelope",
          "question": "Weighing around a ton, the eland is the largest species of this animal in Africa"
        },
        {
          "_additional": {
            "distance": 0.21438634
          },
          "answer": "species",
          "question": "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification"
        },
        {
    

## Vector Databases support for CRUD operations

### Create

In [17]:
#Create an object
object_uuid = client.data_object.create(
    data_object={
        'question': "Leonardo da Vinci was born in this country.",
        'answer': "Italy",
        'category': "Culture"},
    class_name="Question")

In [18]:
print(object_uuid)

7f747366-ea86-4660-9642-b30aed2d4010


### Read

In [19]:
data_object = client.data_object.get_by_id(
    object_uuid, class_name="Question")
json_print(data_object)

{
  "class": "Question",
  "creationTimeUnix": 1701545250508,
  "id": "7f747366-ea86-4660-9642-b30aed2d4010",
  "lastUpdateTimeUnix": 1701545250508,
  "properties": {
    "answer": "Italy",
    "category": "Culture",
    "question": "Leonardo da Vinci was born in this country."
  },
  "vectorWeights": null
}


In [20]:
data_object = client.data_object.get_by_id(
    object_uuid,
    class_name='Question',
    with_vector=True)
json_print(data_object)

{
  "class": "Question",
  "creationTimeUnix": 1701545250508,
  "id": "7f747366-ea86-4660-9642-b30aed2d4010",
  "lastUpdateTimeUnix": 1701545250508,
  "properties": {
    "answer": "Italy",
    "category": "Culture",
    "question": "Leonardo da Vinci was born in this country."
  },
  "vector": [
    0.022491472,
    -0.013062453,
    -0.0031088893,
    -0.047720812,
    -0.0038829134,
    0.012032554,
    -0.017399546,
    -0.0040556295,
    -0.004519404,
    -0.034671154,
    0.0105100935,
    0.025689917,
    -0.0032112396,
    -0.0072348844,
    0.010337377,
    -0.001473684,
    0.01306885,
    0.0053605954,
    0.03953279,
    -0.00806648,
    0.002354056,
    0.021109743,
    0.011699915,
    -0.01088751,
    0.0051399027,
    0.0018854839,
    0.016516775,
    -0.03216357,
    0.014827995,
    -0.031702995,
    -0.007880971,
    0.0006992602,
    -0.009953563,
    -0.0133950915,
    -0.018742893,
    -0.014431388,
    -0.007682667,
    -0.028095149,
    0.00814964,
    0.015301

### Update

In [21]:
client.data_object.update(
    uuid=object_uuid,
    class_name="Question",
    data_object={'answer':"Florence, Italy"})

In [22]:
data_object = client.data_object.get_by_id(
    object_uuid, class_name='Question')
json_print(data_object)

{
  "class": "Question",
  "creationTimeUnix": 1701545250508,
  "id": "7f747366-ea86-4660-9642-b30aed2d4010",
  "lastUpdateTimeUnix": 1701545308785,
  "properties": {
    "answer": "Florence, Italy",
    "category": "Culture",
    "question": "Leonardo da Vinci was born in this country."
  },
  "vectorWeights": null
}


### Delete

In [23]:
json_print(
    client.query.aggregate("Question").with_meta_count().do())

{
  "data": {
    "Aggregate": {
      "Question": [
        {
          "meta": {
            "count": 11
          }
        }
      ]
    }
  }
}


In [24]:
client.data_object.delete(
    uuid=object_uuid, class_name="Question")

In [25]:
json_print(
    client.query.aggregate("Question").with_meta_count().do())

{
  "data": {
    "Aggregate": {
      "Question": [
        {
          "meta": {
            "count": 10
          }
        }
      ]
    }
  }
}
